In [1]:
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.curdir, '..')))
from src.utils import *
from src.ikflow_program import PandaIKProgram
from pydrake.all import (
    StartMeshcat,
    Quaternion,
    RigidTransform
)



ikflow/config.py | Using device: 'cuda:0'


In [2]:
meshcat = StartMeshcat()
diagram = BuildEnv(meshcat=meshcat, directives_file = os.path.join(RepoDir(), "models/panda/panda_collision.yaml"))

program = PandaIKProgram(diagram)
target_pose = np.array([-0.5, 0.5, 0.5, -0.6, 0.48, 0.64, 0]) #x, y, z, qw, qx, qy, qz
program.create_prog(target_pose)


# result = program.Solve()

# result.is_success()



INFO:drake:Meshcat listening for connections at http://localhost:7001


WorldModel::LoadRobot: /home/tangles/.cache/jrl/urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
Geometry: Loading 12 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes/visual/link0.dae into Group
Geometry: Loading 4 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes/visual/link3.dae into Group
Geometry: Loading 4 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes/visual/link4.dae into Group
Geometry: Loading 3 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes/visual/link5.dae into Group
Geometry: Loading 17 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes/visual/link6.dae into Group
Geometry: Loa

In [3]:
result = program.Solve()

print(result.is_success())

vars = result.get_x_val()

True


In [6]:
q = program.VarsToQ(vars)
program.plant.SetPositions(program.plant_context, q)
program.diagram.ForcedPublish(program.diagram_context)


pose = program.target_pose
pose = RigidTransform(Quaternion(pose[3], pose[4], pose[5], pose[6]), [pose[0], pose[1], pose[2]])
DrawAxes(pose, meshcat)


In [ ]:
print("Position Error", program.EvalPositionError(vars))
print("Orientation Error", program.EvalOrientationError(vars)) ### This is a lot


# joint limits

print("lower joint limits", program.plant.GetPositionLowerLimits()[:-2])
print("upper joint limits", program.plant.GetPositionUpperLimits()[:-2])
print("solution joint values", q[:-2])

Position Error [ 2.64799680e-07  2.21930225e-07 -8.08055808e-07]
Orientation Error [0.27113744]
lower joint limits [-2.8973 -1.7628 -2.8973 -3.0718 -2.8973 -0.0175 -2.8973]
upper joint limits [ 2.8973  1.7628  2.8973 -0.0698  2.8973  3.7525  2.8973]
solution joint values [ 2.26196575  0.42782673 -0.03005765 -1.53775442 -1.23310268  3.14793181
  1.77463329]
